In [1]:
from glob import glob

for g in glob('../data/*.pdf'):
    print(g)

../data\2040_seoul_plan.pdf
../data\OneNYC_2050_Strategic_Plan.pdf


In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def read_pdf_and_split_text(pdf_path, chunk_size=1000, chunk_overlap=100):
    """
    주어진 pdf 파일을 읽고 텍스트를 분할합니다.
    매개변수:
        pdf_path (str): PDF 파일의 경로.
        chunk_size (int, 선택적): 각 텍스트 청크의 크기, 기본값은 1000입니다.
        chunk_overlap (int, 선택적): 청크 간의 중첩 크기, 기본값은 100입니다.
    반환값:
        list: 분할된 텍스트 청크의 리스트.
    """
    print(f"PDF: {pdf_path} -----------------------------------")

    pdf_loader = PyPDFLoader(pdf_path)
    data_from_pdf = pdf_loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

    splits = text_splitter.split_documents(data_from_pdf)

    print(f"Number of splits: {len(splits)}\n")

    return splits

In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
import os 

# vectorStore 설정하
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

persist_directory = '../chroma_store'

if os.path.exists(persist_directory):
    print("Loading existing Chroma store")
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding
    )
else:
    print('Creating new Chroma Store')

    vectorstore = None
    for g in glob('../data/*.pdf'):
        chunks = read_pdf_and_split_text(g)
        # 100개씩 나눠서 저장 
        for i in range(0, len(chunks), 100):
            if vectorstore is None:
                vectorstore = Chroma.from_documents(
                    documents=chunks[i:i+100],
                    embedding=embedding,
                    persist_directory=persist_directory
                )
            else:
                vectorstore.add_documents(
                    documents=chunks[i:i+100]
                )

Loading existing Chroma store


In [4]:
retriever = vectorstore.as_retriever(search_kwargs={'k': 5})

chunks = retriever.invoke('서울 온실가스 저감 계획')

for chunk in chunks:
    print(chunk.metadata)
    print(chunk.page_content)

{'creationdate': '2023-02-14T11:05:36+09:00', 'source': 'D:\\projects\\gpt_agent_2025_book\\chap09\\data\\2040_seoul_plan.pdf', 'moddate': '2023-02-14T18:21:42+09:00', 'creator': 'PScript5.dll Version 5.2.2', 'producer': 'Acrobat Distiller 9.0.0 (Windows)', 'title': '', 'total_pages': 272, 'author': '', 'page': 248, 'page_label': '249'}
우려 및 지천의 복원에 대한 검토 필요
∙ 탄소중립을 계획의 주요 원칙으로 삼았으며, 부문별 전략
계획을 통해 자연과 공존하는 도시를 위한 계획 방향을 
설정
- 수변공간 활성화를 위해 단절·훼손된 녹지축과 하천축을 
복원
- 도심 속 생물다양성을 확보하여 친환경 생태기반을 마련
반영
[표 6-2] 2040 서울도시기본계획 공청회 의견 및 조치결과
2. 서울특별시의회 의견청취 결과 : 원안가결
1) 서울특별시의회 본회의 개요
 회의명 : 제314회 임시회 도시계획균형위원회
 일   자 : 2022. 9. 22. (목)
 장   소 : 서울특별시의회 도시계획균형위원회 회의실
 결   과 : 원안가결
6.1 행정절차 상 의견수렴 결과 239
2) 서울특별시의회 의견청취 및 조치결과
주요의견 조치계획 비고
∙ 6대목표에 환경부문을 추가 검토
∙ 7대 목표 중 하나로 ‘탄소중립 안
{'source': 'D:\\projects\\gpt_agent_2025_book\\chap09\\data\\2040_seoul_plan.pdf', 'author': '', 'moddate': '2023-02-14T18:21:42+09:00', 'total_pages': 272, 'creator': 'PScript5.dll Version 5.2.2', 'page_label': '177',

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

model.invoke("안녕하세요")

AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'id': 'chatcmpl-DBuacH2TWV2aOmBNcgcbW4b6j6qdz', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c836d-60b9-7262-920e-00a0134b1aaa-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from typing import Literal # 문자열 리터럴 타입을 지원하는 typing 모듈의 클래스 
from pydantic import BaseModel, Field

# Data Model
class RouteQuery(BaseModel):
    """사용자 쿼리를 가장 관련성이 높은 데이터 소스로 라우팅합니다."""

    datasource: Literal["vectorstore", "casual_talk"] = Field(
        ...,
        description="""
        사용자 질문에 따라 casual_talk 또는 vectorstore로 라우팅합니다.
        - casual_talk: 일상 대화를 위한 데이터 소스, 사용자가 일상적인 질문을 할 때 사용합니다.
        - vectorstore: 사용자 지룸에 답하기 위해 RAG로 vectorstore 검색이 필요한 경우 사용합니다.
        """
    )

In [8]:
# 특정 모델을 structured output(구조화된 출력)과 함께 사용하기 위해 설정 
structured_llm_router = model.with_structured_output(RouteQuery)

router_system="""
당신은 사용자의 질문을 vectorstore 또는 casual_talk으로 라우팅하는 전문가입니다. 
- vectorstore에는 서울, 뉴욕의 발전 계획과 관련된 문서가 포함되어 있습니다. 이 주제에 대한 질문에는 vectorstore를 사용하십시오.
- 사용자의 질문ㅇ이 일상 대화에 관련된 경우에는 casual_talk을 사용하십시오.
"""

# 시스템 메시지와 사용자의 질문을 포함하는 프롬프트 템플릿 생성 
route_prompt = ChatPromptTemplate.from_messages([
    ('system', router_system),
    ('human', "{question}")
])

# 라우터 프롬프트와 구조화된 출력 모델을 결합한 객체 
question_router = route_prompt | structured_llm_router

In [9]:
print(
    question_router.invoke({
        'question': '서울 온실가스 저감 계획은 무엇인가요?'
    })
)

print(question_router.invoke({'question': '잘 지냈어?'}))

datasource='vectorstore'
datasource='casual_talk'
